In [ ]:
'''
Corrective RAG Implementation
Author: Christian Sarmiento
Purpose: This notebook is intended to get a Self-RAG implementation set up with LangChain/LangGraph.
Date Created: 12/4/24
Last Updated: 12/4/24
Data: Marist College Administrative Corpus Dataset
Sources:
- https://blog.langchain.dev/agentic-rag-with-langgraph/
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb?ref=blog.langchain.dev
Note: Most of the code for graph implementation of CRAG was taken from the second source.
-----------------------------------------------------------------------------------------------------------------------
RAG Research             |               Machine Learning Independent Study             |              DR. EITEL LAURIA
'''

In [1]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.


In [22]:
# Imports
import sys
sys.path.append("/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Private Code")
from api_keys import openAIKey
from api_keys import langchainKey
from api_keys import tavilyKey  # web search packages
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import List, Dict, Optional
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_community.tools.tavily_search import TavilySearchResults

import pandas as pd
import os
import gradio as gr  # easy frontend implementation
from pprint import pprint
import numpy as np


from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import SingleTurnSample

In [23]:
# LangChain Enviornment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langchainKey()
os.environ["OPENAI_API_KEY"] = openAIKey()
os.environ["TAVILY_API_KEY"] = tavilyKey()

In [24]:
# Load Data
csvPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/Marist_QA.csv"
maristQA = pd.read_csv(csvPath, header=None)

# To use RecursiveCharacterTextSplitter, we need a list of dictionaries
maristContext = [Document(page_content=text) for text in maristQA[1].tolist()]

In [25]:
# Split Documents into Chunks
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
texts = textSplitter.split_documents(maristContext)

In [26]:
# Store Documents in Vector DB (Chroma)
vectorDB = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

# Setup Retrieval System
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieves 3 documents

In [27]:
# Define Grader class for document grading in CRAG
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llmDocGrader = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMGrader = llmDocGrader.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
   tic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
gradePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Grader Chain
retrievalGrader = gradePrompt | structuredLLMGrader

# Testing it
question = "agent memory"
docs = retriever.get_relevant_documents(question)
docTxt = docs[1].page_content
print(retrievalGrader.invoke({"question": question, "document": docTxt}))

binary_score='no'


In [28]:
# Generation Chain

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
ragChain = prompt | llm | StrOutputParser()

# Run
generation = ragChain.invoke({"context": docs, "question": question})
print(generation)

I don't know.


In [29]:
# Question Rewriter
llmRewriter = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
rewritePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

questionRewriter = rewritePrompt | llm | StrOutputParser()
questionRewriter.invoke({"question": question})

'What are the key concepts and techniques related to agent memory in artificial intelligence?'

In [30]:
# Web Search Component
webSearchTool = TavilySearchResults(k=3)

In [31]:
# Define metric evaluator

## Evaluation LLM & embeddings
evalLLM = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evalEmbeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

## Initialize metrics with LLM and embeddings
contextRecall = LLMContextRecall(llm=evalLLM)
faithfulness = Faithfulness(llm=evalLLM)
factualCorrectness = FactualCorrectness(llm=evalLLM)
semanticSimilarity = SemanticSimilarity(embeddings=evalEmbeddings)

## Collect metrics
evalMetrics = [
    contextRecall,
    faithfulness,
    factualCorrectness,
    semanticSimilarity
]

In [41]:
# Define Graph structure for Self-RAG

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        web_search: whether to add search
        metrics: evaluation metrics for each generation 
    """
    question: str
    generation: str
    webSearch: str
    documents: List[str]
    metrics: Optional[Dict[str, float]]
    visitedInternet: bool

## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "visitedInternet": state["visitedInternet"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "visitedInternet": state["visitedInternet"]}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filteredDocs = []
    webSearch = "No"
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filteredDocs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            webSearch = "Yes"
            continue

    return {"documents": filteredDocs, "question": question, "webSearch": webSearch, "visitedInternet": state["visitedInternet"]}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    betterQuestion = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": betterQuestion, "visitedInternet": state["visitedInternet"]}

def webSearch(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]
    internetFlag = state["visitedInternet"]

    # Web search
    docs = webSearchTool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    internetFlag = True

    return {"documents": documents, "question": question, "visitedInternet": internetFlag}

## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    webSearch = state["webSearch"]
    state["documents"]

    if webSearch == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"
    
async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state

In [42]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery)  
workflow.add_node("webSearchNode", webSearch)
workflow.add_node("evaluateMetrics", evaluateMetrics)  

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate",
    },
)
workflow.add_edge("transformQuery", "webSearchNode")
workflow.add_edge("webSearchNode", "generate")
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("evaluateMetrics", END)

# Compile
app = workflow.compile()

In [16]:
# Inital test run
inputs = {"question": "Who is Carolyn Matheus?"}
async for output in app.astream(inputs):
    for key, value in output.items():

        # Print node
        pprint(f"Node '{key}':")

        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)

        # Print metrics
        if "metrics" in value:
            pprint("Metrics: ")
            pprint(value["metrics"])

    pprint("\n---\n")

# Final generation
pprint("Final Generation: ")
pprint(value["generation"])

# Final metrics
if "metrics" in value:
    pprint("Final Metrics: ")
    pprint(value["metrics"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
'\n---\n'
'Final Generation: '
('Dr. Carolyn C. Matheus is an Associate Professor of Information Systems and '
 'the Director of the Honors Program at Marist College. She holds a PhD in '
 'Organizational Studies with a focus on leadership from SUNY Albany and has '
 'received the National Society of Leadership and Success award for Excellence '
 'in Teaching. Dr. Matheus is involved in faculty-student research projects '
 'and offers seminars on authentic leadership and innovation.')
'Final Metrics

In [65]:
# Put CRAG into Gradio
evaluationSamples = []
ragResults = []
async def correctiveRAG(userQuery, history, correctAnswer=None):
    """
    Gradio-compatible function to process CRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        tuple: (chatDisplay, history)
    """

    # Variables
    inputs = {"question": userQuery, "visitedInternet": False}
    finalOutput = None
    internetFlag = False

    # Start the workflow
    async for output in app.astream(inputs):

        # Saving final output for metric purposes
        finalOutput = output

        # Check if the process went to the internet
        if "webSearchNode" in output:
            internetFlag = True

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Display output for gradio
    #chatDisplay = [(msg["content"], "User" if msg["role"] == "user" else "LLM") for msg in history]

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })
    
    # Save the result of the query with the metrics 
    ragResults.append({
        "question": userQuery,
        "generation": generation,
        "retry": internetFlag
    })

    return history #, chatDisplay

In [ ]:
# Gradio frontend
interface = gr.Interface(
    fn=correctiveRAG,
    inputs=[
        gr.Textbox(label="Ask a Question", placeholder="Enter your question here..."),
        gr.State(),  # Keeps track of conversation history
        gr.Textbox(label="Correct Answer (Optional)", placeholder="For evaluation purposes..."),
    ],
    outputs=[
        gr.Chatbot(label="CorrectiveRAG Conversation"),
        gr.State(),  # Updates conversation history
    ],
    title="CorrectiveRAG Implementation",
    description="Interact with the CRAG workflow for document-grounded question answering.",
)

# Launch the interface
interface.launch()

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}


In [61]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"question": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats

In [44]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [21]:
# Evaluate our pipeline responses
evalResults = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

{'input_query': 'Who is Carolyn Matheus?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.928909234587568}


In [74]:
# Sample 222 records from our dataset
maristTestSample = maristQA.sample(100, replace=False)
maristTestSample.head()

,0,1
586,How to avoid spams,"""\u00a0Impersonation emails\u00a0Robocalls cla..."
587,What is the phone number for student financial...,"""Transfer Student Admission Department Student..."
568,What happens to the funds in my Marist Money a...,"""Student Services Department Card Services Nam..."
103,When is the the office of financial services o...,"""Job Location and Development Information (Fox..."
105,tell me about marist counseling,"""Counseling Services Department Counseling Ser..."


In [ ]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    chatHistory = await correctiveRAG(row[1][0], chatHistory, row[1][1])

## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7596856019922399}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7045918562747744}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUE

The LLM did not return a valid classification.


"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': nan,
 'SemanticSimilarity': 0.7383440103663941}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'gradeDocuments':"
---TRANSFORM QUERY---
"Node 'transformQuery':"
---WEB SEARCH---
"Node 'webSearchNode':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7349868099610156}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL D

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24; trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=5589d19a-9bc9-485e-bc90-ca775f49d5d7; trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=61a27edd-012b-497d-ac66-b1096d2dbae1; patch: trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=cbe9cc23-f9c6-4b37-98e9-7be1960274a6; trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=dabfbcec-c8bb-43c0-82d7-def740475b96; trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=b1101fbd-dbfb-4827-9b48-019d595c0283
Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests f

{'question': 'What GPA do I need for grad school?', 'LLMContextRecall': 1.0, 'FactualCorrectness': 0.11, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8139424641650321}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'LLMContextRecall': 0.16666666666666666, 'FactualCorrectness': 0.14, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8410372116485767}
{'question': 'Who is the chair of the english department?', 'LLMContextRecall': 0.16666666666666666, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.793254676322769}
{'question': 'What are some of the academic resources I can leverage?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.74, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8964862653975958}
{'question': 'Andrew Tokash Classes?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.07, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8689408453581797}
{'question': "Who's the person to go to for Latin American and Caribbean Studies

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=a6fa4e94-d088-49e5-b7ac-e8fbd6522ee9; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=5f062a7b-982d-41fd-88ec-f7d7cf611cad; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=44007c99-0a77-4b1a-9c62-66257e6e8d80; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=870fcd51-3f00-48b9-97d9-ee86222ff501; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=7daee0f7-541e-447d-8eb1-be2e3ed8b89a; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=dd80434a-c0ad-4757-9a11-c87f717ff589; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=06968678-e7f0-42ca-992b-db64dac1144c; trace=e568f6a0-bbcd-45cd

In [76]:
# Make our dataframe of results
processResults = pd.DataFrame(ragResults)
evalResultsDF = pd.DataFrame(evalResults)
finalResultsDF = processResults.merge(evalResultsDF, on="question", how="left")
finalResultsDF

,question,generation,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
0,What GPA do I need for grad school?,You typically need a minimum overall GPA of 3....,False,1.000000,0.11,0.0,0.813942
1,What GPA do I need for grad school?,You typically need a minimum overall GPA of 3....,False,0.500000,0.10,0.0,0.813942
2,Who published Displacements and Transformation...,"The publisher of the book ""Displacements and T...",True,0.166667,0.14,0.0,0.841037
3,Who is the chair of the english department?,The current chair of the English department is...,True,0.166667,0.00,0.0,0.793255
4,What are some of the academic resources I can ...,"You can leverage several academic resources, i...",False,0.000000,0.74,0.0,0.896486
...,...,...,...,...,...,...,...
173,Where is the advising and academic services ce...,The advising and academic services center is l...,False,0.000000,0.29,0.0,0.876235
174,How to use iLearn,"To use iLearn, start by watching the short ""On...",False,0.000000,0.50,0.0,0.911958
175,Janine Peterson job?,Janine Peterson currently works at the Nationa...,True,1.000000,0.00,0.0,0.806813
176,What about Blackridge technology,Blackridge technology focuses on cybersecurity...,True,0.200000,0.00,0.0,0.727944


In [77]:
# Averages & St. devs
print(f"LLM Context Recall - Mean: {finalResultsDF["LLMContextRecall"].mean()}, St. Dev: {finalResultsDF["LLMContextRecall"].std()}")
print(f"FactualCorrectness - Mean: {finalResultsDF["FactualCorrectness"].mean()}, St. Dev: {finalResultsDF["FactualCorrectness"].std()}")
print(f"Faithfulness - Mean: {finalResultsDF["Faithfulness"].mean()}, St. Dev: {finalResultsDF["Faithfulness"].std()}")
print(f"SemanticSimilarity - Mean: {finalResultsDF["SemanticSimilarity"].mean()}, St. Dev: {finalResultsDF["SemanticSimilarity"].std()}")


LLM Context Recall - Mean: 0.3502828980522305, St. Dev: 0.38078152337104065
FactualCorrectness - Mean: 0.229438202247191, St. Dev: 0.21680191980388025
Faithfulness - Mean: 0.0025280898876404497, St. Dev: 0.023930497237328208
SemanticSimilarity - Mean: 0.861302026325832, St. Dev: 0.05792963836449268


In [81]:
final100ResultsDF = finalResultsDF.tail(100)

In [82]:
# Dervive generations that didn't use the internet
trueContextsDF = final100ResultsDF[final100ResultsDF["retry"] == False]
trueContextsDF.describe()

,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
count,55.000000,55.000000,55.0,55.000000
mean,0.374025,0.356000,0.0,0.888085
std,0.395259,0.213772,0.0,0.049715
min,0.000000,0.000000,0.0,0.773360
25%,0.000000,0.195000,0.0,0.857180
50%,0.250000,0.360000,0.0,0.892533
75%,0.650000,0.500000,0.0,0.925828
max,1.000000,0.780000,0.0,0.971669


In [83]:
trueContextsDF.head(100)

,question,generation,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
78,Do I have to research something specific for m...,"No, you do not have to research something spec...",False,0.571429,0.24,0.0,0.779964
82,What is accommodations and accessibility?,Accommodations and accessibility refer to the ...,False,0.000000,0.57,0.0,0.871183
86,How many hours of clinical field experience is...,The Clinical Mental Health Counseling program ...,False,0.250000,0.30,0.0,0.884294
88,Where can you do internships,You can do internships in New York City and th...,False,1.000000,0.50,0.0,0.889595
89,Admissions contact?,You can contact the Office of Graduate Admissi...,False,0.000000,0.00,0.0,0.857182
90,School of Communications internship opportunities,The School of Communication and the Arts offer...,False,0.900000,0.35,0.0,0.919466
91,on what principles is the college's education ...,The college's education is grounded on princip...,False,0.454545,0.19,0.0,0.847089
93,How can I learn about the term abroad program?,"You can learn about the term ""abroad program"" ...",False,0.500000,0.44,0.0,0.898406
94,Dr. Nicholas Marshall,Dr. Nicholas Marshall is an Associate Professo...,False,0.000000,0.55,0.0,0.920979
95,MA clinical mental health,The Master of Arts in Clinical Mental Health C...,False,0.000000,0.48,0.0,0.928664
